In [396]:
import pandas as pd
import numpy as np
from vnstock import *
from datetime import datetime, timedelta
from dateutil.parser import parse
from dateutil.relativedelta import relativedelta


In [397]:
csv_file_path = 'C:/Users/atlas/OneDrive/Desktop/vnstock/daily_price.csv'
daily_price = pd.read_csv(csv_file_path)

In [398]:
daily_price.head(3)

,time,open,high,low,close,volume,ticker
0,2012-03-20,4230,4230,4138,4184,302500,AAA
1,2012-03-21,4207,4461,4207,4392,826800,AAA
2,2012-03-22,4392,4693,4392,4623,672400,AAA


In [399]:
# Convert 'time' to datetime if not already in datetime format
daily_price['time'] = pd.to_datetime(daily_price['time'])

# Group by year and ticker, then select the row with the maximum date within each group
yearly_price = daily_price.groupby([daily_price['time'].dt.year, 'ticker']).apply(lambda x: x.loc[x['time'].idxmin()])

# Reset the index
yearly_price.reset_index(drop=True, inplace=True)

# If you want to keep only specific columns in yearly_price
# yearly_price = yearly_price[['time', 'close', 'ticker']]


In [400]:
yearly_price['next year price'] = yearly_price.groupby('ticker')['close'].shift(-1)


In [401]:
yearly_price['return'] = yearly_price['next year price'] / yearly_price['close'] - 1

In [402]:
yearly_price = yearly_price[['time','close','ticker','next year price','return']]

In [403]:
yearly_price['year'] = yearly_price['time'].dt.year

In [404]:
yearly_price[yearly_price['ticker']=='AAA'].head(2)

,time,close,ticker,next year price,return,year
0,2012-03-20,4184,AAA,4011.0,-0.041348,2012
254,2013-01-02,4011,AAA,5207.0,0.298180,2013


In [405]:

# write df to csv file and open it.
csv_file_path = 'C:/Users/atlas/OneDrive/Desktop/vnstock/yearly_price.csv'
yearly_price.to_csv(csv_file_path, index=False)
# Open the CSV file
os.startfile(csv_file_path)

In [406]:
df = yearly_price

In [407]:
# Sort DataFrame by year
df.sort_values(by=['year'], inplace=True)

# Create an empty list to store DataFrames
combined_list = []

# Group by ticker
grouped = df.groupby('ticker')

# Iterate through each group
for ticker, group in grouped:
    # Iterate through each row in the group
    for index, row in group.iterrows():
        # Calculate average return up to current year excluding the current year
        avg_return_value = group.loc[group['year'] < row['year'], 'return'].mean()
        # Calculate standard deviation of return up to current year excluding the current year
        std_return_value = group.loc[group['year'] < row['year'], 'return'].std()
        # Append the results to the combined list
        combined_list.append(pd.DataFrame({'year': [row['year']], 'ticker': [ticker], 'avg_return': [avg_return_value], 'std_return': [std_return_value]}))

# Concatenate DataFrames in combined_list
df_output = pd.concat(combined_list, ignore_index=True)


In [408]:
df_output[df_output['ticker'] =='AAA'].head()

,year,ticker,avg_return,std_return
0,2012,AAA,NaN,NaN
1,2013,AAA,-0.041348,NaN
2,2014,AAA,0.128416,0.240083
3,2015,AAA,0.111217,0.172358
4,2016,AAA,0.119796,0.141772


In [409]:
merged_df = pd.merge(yearly_price,df_output,left_on=['year','ticker'], right_on = ['year','ticker'],how='left')
merged_df2 = merged_df

In [410]:
merged_df2.rename(columns={'close': 'initial_price', 'avg_return': 'mu', 'std_return': 'sigma'}, inplace=True)

In [411]:
input_df = merged_df2

In [412]:
merged_df2[merged_df2['ticker']=='AAA']

,time,initial_price,ticker,next year price,return,year,mu,sigma
0,2012-03-20,4184,AAA,4011.0,-0.041348,2012,NaN,NaN
419,2013-01-02,4011,AAA,5207.0,0.298180,2013,-0.041348,NaN
683,2014-01-02,5207,AAA,5607.0,0.076820,2014,0.128416,0.240083
974,2015-01-05,5607,AAA,6423.0,0.145532,2015,0.111217,0.172358
1275,2016-01-04,6423,AAA,13551.0,1.109762,2016,0.119796,0.141772
1602,2017-01-03,13551,AAA,19063.0,0.406760,2017,0.317789,0.459435
1961,2018-01-02,19063,AAA,11377.0,-0.403189,2018,0.332618,0.412534
2334,2019-01-02,11377,AAA,10299.0,-0.094753,2019,0.227502,0.468150
2714,2020-01-02,10299,AAA,12869.0,0.249539,2020,0.187220,0.448148
3106,2021-01-04,12869,AAA,20700.0,0.608517,2021,0.194145,0.419718


In [ ]:

# Parameters
initial_price = 29130
  # Initial stock price
mu =0.01  # Drift rate
sigma = 0.02  # Volatility
time_interval = 1/52  # One week time interval (assuming 52 weeks in a year)
num_simulations = 1  # Number of simulations

# Simulate stock price movement
for _ in range(num_simulations):
    stock_price = initial_price
    for _ in range(52):  # 52 weeks in a year
        # Generate random number for Wiener process
        dW = np.random.normal(loc=0, scale=np.sqrt(time_interval))
        
        # Calculate change in stock price
        dS = mu * stock_price * time_interval + sigma * stock_price * dW
        
        # Update stock price
        stock_price += dS
    
    print("Final stock price:", stock_price)


The output "198.1179536728125" represents the simulated final stock price of Apple (AAPL) 
after one year, based on the parameters and simulation method we used.
1. Drift rate (μ): Let's assume an annual average return of 8%. So, =0.08 , μ=0.08 per year.
2. Volatility (σ): Let's assume an annual volatility of 20%. So, =0.20, σ=0.20 per year.

In [414]:
import numpy as np
import pandas as pd

# Define the function to predict next year's stock price
def predict_next_year_price(row):
    initial_price = row['initial_price']
    mu = row['mu']
    sigma = row['sigma']
    time_interval = 1/52  # One week time interval (assuming 52 weeks in a year)
    num_simulations = 1  # Number of simulations
    
    # Simulate stock price movement
    for _ in range(num_simulations):
        stock_price = initial_price
        for _ in range(52):  # 52 weeks in a year
            # Generate random number for Wiener process
            dW = np.random.normal(loc=0, scale=np.sqrt(time_interval))
            
            # Calculate change in stock price
            dS = mu * stock_price * time_interval + sigma * stock_price * dW
            
            # Update stock price
            stock_price += dS
    
    return stock_price



# Apply the function to create the output DataFrame
output_df = input_df.copy()
output_df['predict_price_next_year'] = input_df.apply(predict_next_year_price, axis=1)


In [ ]:
output_df[output_df['ticker']=='VIC']

# write df to csv file and open it.
csv_file_path = 'C:/Users/atlas/OneDrive/Desktop/vnstock/output_df_wiener.csv'
output_df.to_csv(csv_file_path, index=False)
# Open the CSV file
os.startfile(csv_file_path)